In [69]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns
import pygal

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

from scipy.stats import ks_2samp

#pd.set_option("display.max_columns",200)

# Lectura de datos

In [2]:
filename='datos/train.csv'
df=pd.read_csv(filename)
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
filetest='datos/test.csv'
test=pd.read_csv(filetest)
test.head(2)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal


# Ingeniería de datos

In [4]:
df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [5]:
varc=["LotFrontage","LotArea","MasVnrArea","BsmtFinSF1","BsmtFinSF2","TotalBsmtSF",
 "1stFlrSF","2ndFlrSF","LowQualFinSF","GrLivArea","BsmtFullBath","BsmtHalfBath","BsmtUnfSF","FullBath",
 "HalfBath","BedroomAbvGr","KitchenAbvGr","TotRmsAbvGrd","Fireplaces","GarageCars","GarageArea",
 "WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch","PoolArea","MiscVal","MoSold",
 ]
tgt="SalePrice"
id_='Id'

## Análisis de exploratorio e ingenieria de datos

### Tratamiento de variables categoricas y fechas

A todas las variables que el diccionario indica que sus datos ausente representan en realidad
la ausencia de esa caracterisica los rellenamos con un valor en forma de string para poder trabajar con ellos.

In [6]:
df["Alley"].fillna(value='No',inplace=True)
df["BsmtQual"].fillna(value='No',inplace=True)
df["BsmtCond"].fillna(value='No',inplace=True)
df["BsmtExposure"].fillna(value='No',inplace=True)
df["BsmtFinType1"].fillna(value='No',inplace=True)
df["BsmtFinType2"].fillna(value='No',inplace=True)
df["FireplaceQu"].fillna(value='No',inplace=True)
df["GarageType"].fillna(value='No',inplace=True)
df["GarageFinish"].fillna(value='No',inplace=True)
df["GarageQual"].fillna(value='No',inplace=True)
df["GarageCond"].fillna(value='No',inplace=True)
df["PoolQC"].fillna(value='No',inplace=True)
df["Fence"].fillna(value='No',inplace=True)
df["MiscFeature"].fillna(value='No',inplace=True)
df["MasVnrType"].fillna(value='No',inplace=True)

En la variable MasVnr hay dos tipos de valores ausentes que significan lo mismo.

In [56]:
df["MasVnrType"].replace({'None':'No'},inplace=True)

Los valores 'YearBuilt', 'YearRemodAdd' los sustituiremos por su equivalente en antiguedad y
antiguedad de remodelación suponiendo que estamos en el 2011, pues la última venta fue
hecha en el 2010.

In [7]:
df['Antiguedad']=2011-df["YearBuilt"]
df["Antiguedad_rem"]=2011-df["YearRemodAdd"]
df["Antig_Garage"]=2011-df["GarageYrBlt"]
df.drop("GarageYrBlt",axis=1,inplace=True)
df.drop(["YearBuilt","YearRemodAdd"],axis=1,inplace=True)
df.head(2)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,Antiguedad,Antiguedad_rem,Antig_Garage
0,1,60,RL,65.0,8450,Pave,No,Reg,Lvl,AllPub,...,No,0,2,2008,WD,Normal,208500,8,8,8.0
1,2,20,RL,80.0,9600,Pave,No,Reg,Lvl,AllPub,...,No,0,5,2007,WD,Normal,181500,35,35,35.0


In [8]:
varc=varc+["Antiguedad","Antiguedad_rem","Antig_Garage"]

Ahora vamos a crear una lista con las variables discretas.

In [9]:
vard=[]
for col in df.columns:
    if( (col not in varc) and col!=tgt and col != id_):
        vard.append(col)

### Valores ausentes categoricas y OneHotEncoder

Vemos que solo hay una variable con un dato ausente.

In [10]:
df[vard].shape,df[vard].dropna().shape

((1460, 47), (1459, 47))

In [11]:
df[["Electrical"]][df["Electrical"].isna() == True]

,Electrical
1379,NaN


In [12]:
df.drop(1379,axis=0,inplace=True)
df=df.reset_index(drop=True).copy()

Ahora convertiremos las variables discretas en continuas.

In [72]:
aux = df[vard].copy()
oh = OneHotEncoder()
oh.fit(aux)
X = pd.DataFrame(oh.transform(aux).toarray(),columns=oh.get_feature_names())

In [74]:
X=X.assign(Id=df[id_].values)

### Valores ausentes continuas

Notamos que solo tres variables tienen datos ausentes, más aún, solo dos acaparán la gran mayoría de ellos.

In [41]:
aux=(1-(len(df)-df[varc].describe().T[['count']])/len(df))
aux[aux['count']<1]

,count
LotFrontage,0.822481
MasVnrArea,0.994517
Antig_Garage,0.944483


Esta variable la llenaremos con 0 por que las otras variables que tenian None estabán llenas con cero
indicando que no tienen esa area.

In [63]:
df['MasVnrArea'].fillna(value=0,inplace=True)

Para el caso de la antiguedad del Garage no podemos hacer esto, por que notamos que todos los ausentes se deben a que no hay garage, así que las posibilidades que tenemos son la imputación o bien eliminar las 81 filas.

In [66]:
df[df['GarageFinish']=='No']["Antig_Garage"]

39     NaN
48     NaN
78     NaN
88     NaN
89     NaN
        ..
1349   NaN
1406   NaN
1448   NaN
1449   NaN
1452   NaN
Name: Antig_Garage, Length: 81, dtype: float64

En el caso de LotFrontage no es claro a que se debe su ausencia por lo que recurriremos a la imputación.

In [67]:
aux=df[varc]
im = SimpleImputer(strategy='median')
im.fit(aux)
Xc = pd.DataFrame(im.transform(aux),columns=varc)

En la prueba de bondad de ajuste vemos que ninguna variable se rompió.

In [71]:
#Prueba de Kolmogorov-Smirnov (Prueba de bondad de ajuste que verifica que dos distribuciones son estadísticamente iguales)k
ks = pd.DataFrame(map(lambda v: (v,ks_2samp(aux[v].dropna(),Xc[v]).statistic),varc),columns=['variable','ks'])
ks.loc[ks['ks']>.1]

,variable,ks


In [75]:
Xc=Xc.assign(Id=df[id_].values)

In [87]:
tad = Xc.merge(X,on=id_,how='inner')

In [92]:
varc=list(tad.columns)

In [93]:
tad=tad.assign(SalesPrice=df[tgt].values)

### Unarias

In [94]:
unarias = [v for v,n in [(v,tad[v].unique().shape[0]) for v in varc] if n ==1 ]
unarias

[]

### Valores extremos

In [105]:
limites=tad.describe(percentiles=[0.01,0.99]).T[['1%','99%']].reset_index().values.tolist()

In [107]:
for v,li,ls in limites:
    tad['ol_{}'.format(v)]= ((tad[v]<li)|(tad[v]>ls)).astype(int)

In [109]:
varo = [v for v in tad.columns if v[:2]=='ol']
varo

['ol_LotFrontage',
 'ol_LotArea',
 'ol_MasVnrArea',
 'ol_BsmtFinSF1',
 'ol_BsmtFinSF2',
 'ol_TotalBsmtSF',
 'ol_1stFlrSF',
 'ol_2ndFlrSF',
 'ol_LowQualFinSF',
 'ol_GrLivArea',
 'ol_BsmtFullBath',
 'ol_BsmtHalfBath',
 'ol_BsmtUnfSF',
 'ol_FullBath',
 'ol_HalfBath',
 'ol_BedroomAbvGr',
 'ol_KitchenAbvGr',
 'ol_TotRmsAbvGrd',
 'ol_Fireplaces',
 'ol_GarageCars',
 'ol_GarageArea',
 'ol_WoodDeckSF',
 'ol_OpenPorchSF',
 'ol_EnclosedPorch',
 'ol_3SsnPorch',
 'ol_ScreenPorch',
 'ol_PoolArea',
 'ol_MiscVal',
 'ol_MoSold',
 'ol_Antiguedad',
 'ol_Antiguedad_rem',
 'ol_Antig_Garage',
 'ol_Id',
 'ol_x0_20',
 'ol_x0_30',
 'ol_x0_40',
 'ol_x0_45',
 'ol_x0_50',
 'ol_x0_60',
 'ol_x0_70',
 'ol_x0_75',
 'ol_x0_80',
 'ol_x0_85',
 'ol_x0_90',
 'ol_x0_120',
 'ol_x0_160',
 'ol_x0_180',
 'ol_x0_190',
 'ol_x1_C (all)',
 'ol_x1_FV',
 'ol_x1_RH',
 'ol_x1_RL',
 'ol_x1_RM',
 'ol_x2_Grvl',
 'ol_x2_Pave',
 'ol_x3_Grvl',
 'ol_x3_No',
 'ol_x3_Pave',
 'ol_x4_IR1',
 'ol_x4_IR2',
 'ol_x4_IR3',
 'ol_x4_Reg',
 'ol_x5_Bnk',


In [112]:
#Captura las filas con valores extremos
tad['ext'] = tad[varo].max(axis=1)

In [118]:
tad[varo].sum()

ol_LotFrontage    15
ol_LotArea        22
ol_MasVnrArea     15
ol_BsmtFinSF1     15
ol_BsmtFinSF2     15
                  ..
ol_x46_Alloca     12
ol_x46_Family      0
ol_x46_Normal      0
ol_x46_Partial     0
ol_SalesPrice     30
Length: 338, dtype: int64

In [116]:
len(tad)

1459